Developing Real-Time Capable Controllers at Low Cost Using Simulation
==========================================================

<center>
  <i>Mike Moore</b><br>
  <i>James Holley</b><br>
  <i>Josh Sooknanan</b><br>
</center>

# Abstract

Simulators are useful tools that enable a reduction in development costs for robotic controllers. A variety of open-source simulation packages are readily available and can be used to develop high fidelity models with accompanying 3D graphics. When developed correctly, these simulators can stand in as a substitute for real hardware. This allows the controls developer to employ a lower cost, software focused, and test-driven approach towards developing their control logic. This saves them from the risk of damaging expensive hardware and also reduces the cost and development time. This paper details a case study in which this approach is employed to develop a low cost real-time controller for an unstable robotic system. Bobble-Bot is a custom built, two wheeled, self-balancing robot. It employs an embedded Real-Time Linux OS running a high bandwidth control loop. This paper starts with a derivation of the underlying equations of motion for the system. Next, a survey of available simulation tools is provided including: Python with NumPy/SciPy, Matlab Simulink, ROS with Gazebo, and the Unreal Engine. Simulations of increasing fidelity are then developed and used in order to test Bobble-Bot's controller. Finally, the source code and samples of simulation results are provided.


<style>
img {
  filter: drop-shadow(4px 4px 3px grey);
  align: center;
}
</style>
<center>
<img src="imgs/BobbleCAD.png" alt="BobbleBot CAD" style="width: 300px;"/>
</center>

# Background and Motivation

Bobble-Bot is a modern take on a classical problem in control theory. The robot represents
a unique solution to the well understood problem of control of an inverted pendulum. This
subject has a rich history in both academia and industry. Practical applications of this sort of
control system include: robots in domestic and industrial use, control of rockets and missiles,
and object transport utilizing drones [1]. Though the theory is well understood, an imple-
mentation of such a robot is anything but a trivial endeavor. The Bobble-Bot control system
requires coordination between several important systems – power, sensors, control, and the
communications subsystems are all critical for maintaining the robot's balance. The next sections will briefly summarize the design of the robotic system.


## Hardware Components 
The major hardware components used by Bobble-Bot are shown in the image below. The Raspberry
Pi (RPi) was selected primarily because it was a cheap open-hardware Linux computer that could easily run the Robot Operating System. Brushless DC motors were used because of their advantages in precision speed control, low power consumption, and durability. CAN hardware was selected in order to facilitate sending command and status packets to and from the motor controllers at a high rate (1 Mbps). A LiPo battery and PDU was selected and supplies enough power to enable three hours of continuous operation. The BNO055 IMU [2] was selected as an affordable and complete orientation sensor solution. The sensors is capable of providing a sufficiently accurate tilt and heading angle. This particular IMU requires only a minimal level of driver software development in order to integrate into the selected control system software architecture. The AS5047D [3] absolute position sensor was selected in order to provide an accurate motor position and velocity reading.

<style>
img {
  filter: drop-shadow(4px 4px 3px grey);
}
</style>

<img src="imgs/HwList1.png" alt="BobbleBot CAD" style="width: 800px;"/>
<img src="imgs/HwList2.png" alt="BobbleBot CAD" style="width: 800px;"/>

## Software Modules
The task of balancing Bobble-Bot is achieved by a control algorithm that implements a linear control law. This control law describes a mathematical equation that relates system errors to motor torque commands. The equation can be implemented within a single function provided the error values can be represented faithfully, and the motor torque commands can be executed reliably. The whole system must run in a loop so that the control law can be executed and rapidly update the commands to the motor before the robot tips over.

The critical software tasks can be divided up into modules that are responsible for the following tasks: control law implementation, robot state management, orientation sensing, command packaging, receiving motor status data, encoding communication frames, decoding communication frames, packet transmission, receiving CAN frames, and command handling and data routing. There are many other smaller modules that are also needed for signal processing, data exchange, logging, error handling, and miscellaneous math libraries to list just a few. It is also important to realize that the Bobble-Bot control software is written on-top of a large set of open-source modules available within the Linux and ROS software stack. It is a fairly large set of software modules that must be organized and managed.

<style>
img {
  filter: drop-shadow(4px 4px 3px grey);
}
</style>
<img src="imgs/BB_Sys_Arch.png" alt="BobbleBot System Architecture" style="width: 600px;"/>

## Balancing Before Simulation Development
The video below comes from an early hardware based test of Bobble-Bot's controller logic. At the time this video was taken, the focus of Bobble-Bot development had been on assemblying the hardware and wiring together all of the components. The controller logic was prototyped and demonstrated on the new hardware, and this video was recorded. The video shows the difficulty in proceeding with controller software development using a solely hardware based testing approach.

In [13]:
from IPython.display import HTML
# Youtube
HTML('<center><iframe width="800" height="385" src="https://www.youtube.com/embed/6OErQsLiVfQ" frameborder="0" allow="autoplay; encrypted-media" allowfullscreen></iframe>')


## Why Simulate?
The control jitter seen in the video above needs to be eliminated and the controller software needs futher tuning. We would like to do this in a way that doesn't risk accidentally toppling Bobble-Bot over and damaging the hardware or wiring. Damage of that sort would cause significant delay in the development schedule. One way to solve the problem is to develop a simulation so that controller testing can be done using a purely software based approach. Of course, this immediately eliminates the risk of damaging hardware. Additionally, it allows separate developers to simultaneously implement and test their control algorithms using their own instance of the Bobble-Bot simulator. These are two big advantages that ultimately reduce the development cost and schedule. So how do we develop a simulation of Bobble-Bot? As with many tasks in engineering, it is helpful to first gain an understanding of the system dynamics using a mathematical model.

# Derivation of Math Model
This section derives the Bobble-Bot state space representation by linearizing the equations of motion about a balance point. This is done not only as an excercise in understanding the unstable and non-linear dynamics of the Bobble-Bot system, but also to serve as a basis for implementing prototype simulations in an effort to select viable control system gains using modern control theory. The derivation can be conveniently split into two parts by the introduction of internal forces. Px and Py capture the forces imparted on the pendulum body at the pendulum/wheel joint. As will be seen as the derivation progresses, these forces are solved for and used to combine the equations derived from the two separate pendulum and wheel free-body diagrams. We end up with equations of motion that are in terms of the torque applied by the motors, pendulum length, mass, inertia, and wheel diameter. 

## Pendulum Dynamics
Start with a free body diagram of the pendulum.

<style>
img {
  filter: drop-shadow(4px 4px 3px grey);
}
</style>
<img src="imgs/PendulumDynamics.png" alt="Pendulum Dynamics" style="width: 400px;"/>

The position of the center of mass can be expressed as

$$ r = x \uvec{i} - Lsin(\theta) \uvec{i} + Lcos(\theta) \uvec{j} $$

Differentiating the above twice gives the acceleration of the center of mass

$$ \ddot{r} = (\ddot{x}cos\theta - \ddot{\theta}L)(cos\theta\uvec{i} + sin\theta\uvec{j}) - (\ddot{x}sin\theta+\dot{\theta^2}L)(cos\theta\uvec{j} - sin\theta\uvec{i} ) $$

Summing forces in the x and y direction and combining the expression results in the following
second-order differential equation describing the translational motion of Bobble-Bot's mass center.

$$ m_r(\ddot{x}cos\theta - \ddot{\theta}L) = -m_rgsin\theta + P_ysin\theta + P_xcos\theta $$

Likewise, we sum the torques about the center of mass in order to get another second order differential
equation this time describing the rotational motion about the Bobble-Bot mass center.

$$ I_r\ddot{\theta} = -\tau +  P_yLsin\theta + P_xLcos\theta  $$

Combining the linear and rotational equations of motion we get the following for the pendulum dynamics

$$  -(m_rLcos\theta)\ddot{x} + (I_r + m_rL^2)\ddot{\theta} = m_rgLsin\theta - \tau $$

## Wheel Dynamics
Start with a free body diagram of one of the wheels.

<style>
img {
  filter: drop-shadow(4px 4px 3px grey);
}
</style>
<img src="imgs/WheelDynamics.png" alt="Pendulum Dynamics" style="width: 400px;"/>

Summing the forces in the x-direction gives

$$ m_w\ddot{x} = -P_x - F $$

Likewise, summing the torques about the wheel center gives

$$ I_w\ddot{\phi} = \tau - Fr_w $$

Combining the above two equations and solving for Px we get

$$ P_x = -m_w\ddot{x} + \frac{I_w\ddot{\phi} - \tau}{r_w} $$

The force in the x direction, F, is known from the previous derivation of the pendulum dynamics. 
Substituting that force in and eliminating Px by combining the linear and rotational wheel dynamics
we get the following

$$ m_r(\ddot{x} - \ddot{\theta}Lcos\theta + \dot{\theta}^2Lsin\theta) = -m_w\ddot{x} + \frac{I_w\ddot{\phi} - \tau}{r_w} $$

Rearranging the above gives the following second order non-linear differential equation describing the wheel dynamics

$$ I_w\ddot{\phi} - ( m_rr_w + m_wr_w )\ddot{x} + (m_rr_wLcos\theta)\ddot{\theta} = m_rr_w\dot{\theta}^2Lsin\theta + \tau $$ 


## Linear Assumptions
In order to apply the theory of linear systems and controls, we would like to linearize the equations
derived in the previous section. This allows us to develop a useful linear model which approximates
the real Bobble-Bot system dynamics in the vicinity of its balance point. The following assumptions and simplifications allow us to linearize the equations of motion from the previous section.

Small angle approximation

$$ \theta^2 \approx 0 , sin\theta \approx \theta , cos\theta \approx 1 $$

No slip

$$ \ddot{x} = - r_w\ddot{\phi} $$

Ideal motor

$$ \tau = \frac{2k_tV}{r_m} - \frac{2k_tk_v\dot{\phi}}{r_m}$$


## State Space Representation

Representing our system as a set of differential equations is a bit cumbersome, and it is not immediately useful for most simulation tools. The state space representation of a system replaces an n<sup>th</sup> order differential equation with a single first order matrix differential equation. The state space representation of a system is depicted below [4].

<img src="imgs/StateSpace.png" alt="State Space" style="width: 500px;"/>

<img src="imgs/StateSpace2.png" alt="State Space" style="width: 500px;"/>

The first equation for $\dot{x}$ is called the state equation. The second equation for $y$ is called the output equation.  For an n<sup>th</sup> order system with r inputs and m outputs, the size of each of the matrices is as follows:

 * x is nx1 (n rows by 1 column) and is called the state vector. It is a function of time.
 * A is nxn and is called the state matrix. It is constant.
 * B is nxr and is called the input matrix. It is constant
 * u is rx1 and is called the control input. It is a function of time.
 * C is mxn and is called the output matrix. It is constant
 * D is mxr and is called the feedthrough matrix. It is constant.
 * y is mx1 and is called the ouput. It is a function of time.

To use this form for Bobble-Bot, we first define the following system constants

$$ \alpha = r_m(m_rL^2+I_r) $$
$$ \beta = r_m(I_r + m_rr_w^2+m_wr_w^2) $$
$$ \gamma = m_rLr_m $$

Next, we apply the linearizing assumptions of section 4.3 to the equations of motion derived in sections 4.1 and 4.2. This results in a system of second order linear differential equations that can be encoded in state space form as follows

<img src="imgs/StateSpaceBobbleBot.png" alt="State Space" style="width: 500px;"/>

Now that we have the state space form of our system, we are ready to use a variety of simulation tools. These tools all use numerical integration in order to propagate the system state over time from a known set of initial conditions. The following section explores some options for how one might proceed.


# Options for Simulation
There are many simulation tools available to developers. The table below is certainly not an exhaustive list, however, it summarizes the tools that were considered for this particular project. All of these tools were used in some fashion during the development of Bobble-Bot. Gazebo was the simulation framework chosen for the primary Bobble-Bot simulation. This was done because of its support for ROS control (which is employed on the hardware). The Bobble-Bot Gazebo simulation is capable of running with hardware in the loop. This makes it an invaluable tool for testing out the controller in both software and hardware as it was being developed. It should be noted that Matlab Simulink is also capable of this, but it was deemed cost prohibitive for this application.

<div id="demo">
<table id="table" class="table table-hover table-mc-light-blue">
  <thead>
    <tr>
      <th>Tool</th>
      <th>Pros</th>
      <th>Cons</th>
      <th>Sample Projects</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td data-title="Simulation Tool">
          <a href="https://www.python.org/" target="_blank">Python</a>
      </td>
      <td data-title="Pros">Free, easy to use, lots of 3rd party libraries, rapid prototyping</td>
      <td data-title="Cons">Limited in sim specific features, lacking 3D sim visualization tools</td>
      <td data-title="Sample Project">
        <a href="http://www.github.com/toddsifleet/inverted_pendulum" target="_blank">Cart Pendulum</a>
      </td>
    </tr>
    <tr>
      <td data-title="Simulation Tool">
          <a href="https://www.mathworks.com/products/simulink.html" target="_blank">Matlab Simulink</a>
      </td>
      <td data-title="Pros">Feature rich, many example projects, good support, built in analysis tools</td>
      <td data-title="Cons">Costly, closed source, feature bloat, steep learning curve</td>
      <td data-title="Sample Project">
        <a href="https://www.mathworks.com/matlabcentral/fileexchange/19147-nxtway-gs-self-balancing-two-wheeled-robot-controller-design" target="_blank">Self-Balancing Robot</a>
      </td>
    </tr>
    <tr>
      <td data-title="Simulation Tool">
          <a href="http://gazebosim.org/" target="_blank">Gazebo</a>
      </td>
      <td data-title="Pros">Free, open-source, many example projects, decent 3D visualization, strong ROS support</td>
      <td data-title="Cons">Buggy, no built in analysis tools, limited cross platform support </td>
      <td data-title="Sample Projects">
        <a href="https://github.com/jhu-lcsr/orocos_barrett" target="_blank">Robotic Arm Sim</a>
      </td>
    </tr>
    <tr>
      <td data-title="Simulation Tool">
          <a href="https://www.unrealengine.com/en-US/what-is-unreal-engine-4" target="_blank">Unreal Engine</a>
      </td>
      <td data-title="Pros">Free, cutting edge graphics, NVIDIA PhysX engine, VR support</td>
      <td data-title="Cons">Gaming engine first, steep learning curve, caters more to artists</td>
      <td data-title="Sample Projects">
        <a href="https://docs.unrealengine.com/en-us/Engine/Physics" target="_blank">Simple Examples</a> <br>
        <a href="https://github.com/Microsoft/AirSim" target="_blank">Microsoft AirSim</a> <br>
        <a href="https://www.pluralsight.com/courses/creating-flight-simulator-unreal-engine-2117" target="_blank">Space Flight Simulator</a>
      </td>
    </tr>
  </tbody>
</table>

## Simple 2D Python Sim
Python is a general purpose, interpreted programming language with a wide variety of applications. It is a great language for quick prototyping and experimentation. [NumPy](http://www.numpy.org/) and [SciPy](https://www.scipy.org/) are widely used free Python libraries for engineering and scientific computation. These libraries are useful for verifying the equations of motion derived in the previous section. Here's a numpy and scipy based simulation of a double pendulum. This serves as a good example of how to implement a simple 2D simulation from state equations represented in state space form. [Matplotlib](https://matplotlib.org/) is used to animate the results.

In [ ]:
"""
Inverted pendulum simple Python 2D simulation
State space form derived in M. Moore paper found here:
http://TBD
"""
from numpy import sin, cos
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate as integrate
import matplotlib.animation as animation

class InvertedPendulumSim(object):
    def __init__(self, G=9.8, L1=1.0, L2=1.0, M1=1.0, M2=1.0):
        self.G = G
        self.L1 = L1
        self.L2 = L2
        self.M1 = M1
        self.M2 = M2
        self.fig = plt.figure()
        self.ax = self.fig.add_subplot(111, autoscale_on=False, xlim=(-2, 2), ylim=(-2, 2))
        self.ax.grid()
        self.line, = self.ax.plot([], [], 'o-', lw=2)
        self.time_template = 'time = %.1fs'
        self.time_text = self.ax.text(0.05, 0.9, '', transform=self.ax.transAxes)

    def define_state_derivs(self, state, t):
        dydx = np.zeros_like(state)
        dydx[0] = state[1]

        del_ = state[2] - state[0]
        den1 = (self.M1 + self.M2)*self.L1 - self.M2*self.L1*cos(del_)*cos(del_)
        dydx[1] = (self.M2*self.L1*state[1]*state[1]*sin(del_)*cos(del_) +
                 self.M2*self.G*sin(state[2])*cos(del_) +
                 self.M2*self.L2*state[3]*state[3]*sin(del_) -
                 (self.M1 + self.M2)*G*sin(state[0]))/den1

        dydx[2] = state[3]

        den2 = (self.L2/self.L1)*den1
        dydx[3] = (-self.M2*self.L2*state[3]*state[3]*sin(del_)*cos(del_) +
                 (self.M1 + self.M2)*self.G*sin(state[0])*cos(del_) -
                 (self.M1 + self.M2)*self.L1*state[1]*state[1]*sin(del_) -
                 (self.M1 + self.M2)*self.G*sin(state[2]))/den2
        return dydx

    def simulate(self, initial_state, dt=0.05, sim_duration=20.0):
        self.sim_duration = sim_duration
        self.initial_state = initial_state
        self.time_step_secs = dt
        # create a time array from 0..100 sampled at 0.05 second steps
        t = np.arange(0.0, self.sim_duration, self.time_step_secs)
        # th1 and th2 are the initial angles (degrees)
        # w10 and w20 are the initial angular velocities (degrees per second)
        th1 = 120.0
        w1 = 0.0
        th2 = -10.0
        w2 = 0.0
        # initial state
        state = np.radians(self.initial_state)
        # integrate your ODE using scipy.integrate.
        y = integrate.odeint(self.define_state_derivs, state, t)
        x1 = self.L1*sin(y[:, 0])
        y1 = -self.L1*cos(y[:, 0])
        x2 = self.L2*sin(y[:, 2]) + x1
        y2 = -self.L2*cos(y[:, 2]) + y1
        output_state = [state, y, x1, x2, y1, y2]
        return output_state

    def animate(self, output_state):
        return animation.FuncAnimation(self.fig, self.draw_animation_frame, np.arange(1, len(output_state[1])),
                                       fargs=(self.time_step_secs, output_state), 
                                       interval=25, blit=True, init_func=self.init_plot)

    def draw_animation_frame(self, i, dt, output_state):
        thisx = [0, output_state[2][i], output_state[3][i]] # 0, x1, x2
        thisy = [0, output_state[4][i], output_state[5][i]] # 0, y1, y2    
        self.line.set_data(thisx, thisy)
        self.time_text.set_text(self.time_template % (i*dt))
        return self.line, self.time_text

    def init_plot(self):
        self.line.set_data([], [])
        self.time_text.set_text('')
        return self.line, self.time_text

#########################################################        
# Define simulation configuration terms
#########################################################        
time_step_secs = 0.05
sim_length_secs = 30
#########################################################        
# Define system constants
#########################################################        
G = 9.8  # acceleration due to gravity, in m/s^2
L1 = 1.0 # length of pendulum 1 in m
L2 = 1.0 # length of pendulum 2 in m
M1 = 1.0 # mass of pendulum 1 in kg
M2 = 1.0 # mass of pendulum 2 in kg
#########################################################        
# Define the initial state
#########################################################        
initial_state = [ 120.0, # th1
                    0.0, # w1
                   10.0, # th2
                    0.0  ]
#########################################################        
# Run the simulation
#########################################################        
sim = InvertedPendulumSim(G=G, L1=L1, L2=L2, M1=M1, M2=M2)
output_state = sim.simulate(initial_state, dt=time_step_secs, sim_duration=sim_length_secs)
#########################################################        
# Create the animation from simulation outputs
#########################################################        
anim = sim.animate(output_state)
from IPython.display import HTML
HTML(anim.to_jshtml())

## Matlab Simulink
MATLAB and Simulink work together to combine textual and graphical programming into an integrated simulation environment. Developers can immediately gain access to thousands of libraries containing algorithms authored for MATLAB. This allows the developer to focus on their domain specific application. Users can combine Simulink library blocks with their own custom created blocks that wrap their application specific MATLAB code. This allows users to rapidly build up system simulators by dragging and dropping blocks from reusable component libraries. The full power of MATLAB is also available to Simulink users to analyze the output of their simulations.

As an example of using Matlab Simulink to design a Bobble-Bot like system, refer to the controller design of the [NXTway-GS](https://www.mathworks.com/matlabcentral/fileexchange/19147-nxtway-gs-self-balancing-two-wheeled-robot-controller-design) by Yorihisa Yamamoto. This simulation utilizes Simulink to design and auto generate the controller code. The image below shows that it also comes with a 2D visualization tool.

<style>
img {
  filter: drop-shadow(4px 4px 3px grey);
}
</style>
<img src="imgs/NXTwaySimulink.png" alt="Pendulum Dynamics" style="width: 800px;"/>



## Gazebo
Gazebo is an open-source 3D dynamic simulator that is commonly used to simulate robotic systems. While Gazebo's development environment has the look and feel of a 3D game engine, Gazebo offers physics simulation at a much higher fidelity than is typical of most video games. In addition, Gazebo also contains a large library of open-source robot, sensor, and effector models. Gazebo is the recommended simulation framework for developing and testing control algorithms developed for ROS.


The following list highlights some key features

 * Library of robot models and environments

 * 3D development environment
 
 * Multiple physics engines
 
 * Support for ROS controllers
 
 * Understands URDF (ROS robot model description format)
 
 * Library of sensor models
 
For the reasons above, Gazebo was selected as the simulation framework of choice for developing a high-fidelity Bobble-Bot simulator. This simulation is used to tune the Bobble-Bot controller gains and test out higher level control logic. Future applications include enabling the development and test of [SLAM](https://pdfs.semanticscholar.org/44ca/eb172d89099c184031da6c163c4edd7c8671.pdf) related algorithms and sensor modeling. The Bobble-Bot Gazebo simulator is also capable of running with hardware in the loop. The image below is a screenshot of the Bobble-Bot Gazebo simulation environment.

<img src="imgs/BobbleBotSim_ScreenShot.png" alt="BobbleBot Gazebo Sim"/>


## Unreal Engine 4
Unreal Engine 4 is a suite of integrated tools for developers to design and build games, simulations, and visualizations. The engine is [open-source](https://www.unrealengine.com/en-US/ue4-on-github) and free to download, but developers should check the [licensing agreement](https://www.unrealengine.com/en-US/eula) if they are using it to develop a commercial product. Unreal Engine is still relatively new on the scene for 3D simulation development. The engine relies on NVIDIA's [PhysX](https://developer.nvidia.com/physx-sdk) physics engine. Some simple example physics simulations and tutorials can be found [here](https://docs.unrealengine.com/en-us/Engine/Physics). 

Microsoft's research laboratory has [recently used](https://www.microsoft.com/en-us/research/wp-content/uploads/2017/07/1705.05065.pdf) Unreal Engine to build a high fidelity drone and self-driving car simulator. [AirSim](https://www.microsoft.com/en-us/research/project/aerial-informatics-robotics-platform/) is an open-source, high-fidelity physics, and photo-realistic robot simulator. Microsoft built the simulation framework on top of Unreal Engine in order to help verify control and perception software for robot designers and developers.

Amazing 3D graphics is where Unreal Engine really shines. Developers can achieve Hollywood-quality visuals out of the box. Unreal Engine’s physically-based rendering, advanced dynamic shadow options, screenspace reflections and lighting channels provide the flexibility and efficiency to create awe-inspiring content. The videos below show BobbleBot being rendered in a garage scene using the Unreal Engine.


In [ ]:
from IPython.display import HTML

# Youtube
HTML('<center><iframe width="800" height="500" src="https://www.youtube.com/embed/n23oYdBKZzg" frameborder="0" allow="autoplay; encrypted-media" allowfullscreen></iframe></iframe>')


In [ ]:
from IPython.display import HTML

# Youtube
HTML('<center><iframe width="800" height="500" src="https://www.youtube.com/embed/hooCl_V9Kas" frameborder="0" allow="autoplay; encrypted-media" allowfullscreen></iframe>')


# Simulation Development and Results

Gazebo was selected as the framework of choice for the high fidelity, hardware in the loop, Bobble-Bot simulator. Gazebo is free, open-source, and provides direct support for ROS control. ROS control is the controller architecture that was adopted for Bobble-Bot. One of the key advantages of this architecture is that it provides carefully designed software abstraction layers that enable running real-time controllers with either simulation or hardware in the loop. The controller is agnostic about whether it is running with a simulator in the loop or the real system. This approach allows developers to iron out any lingering bugs and tune their control algorithms using simulation. This is a less risky proposition than developing directly on real hardware with the risk of imposing damage to people or property. This architecture is depicted in the diagram below [5].

<img src="imgs/Gazebo_ros_transmission.png" alt="Sim Architecture"/>

The simulator includes a model of the mass properties (center of gravity and inertia matrix), the wheel friction, motor joints, gravity, and tilt sensor. When the simulator runs without the controller in the loop, the Bobble-Bot model immediately topples over. This is just like in real life. Bobble-Bot requires active control. It is only when the balance controller is ran alongside the simulator, can simulated Bobble-Bot maintain its balance. One of the nice advantages of the simulator is that it can be used to tune the Bobble-Bot PID control gains without the risk of damaging hardware. To do this we were able to run many separate impulse force tests within the simulated environment. These tests involve applying a force of 200-500 Newtons over a tenth of a second time step. This results in an impulse force of 4.5-11.25 lbf*s. This gives the simulated Bobble-Bot a jolt not unlike the one shown in the real hardware testing videos. After that we record the controlled response, manually adjust our control gains, and repeat the process until we get the desired response. Some plots from these simulated runs for different controller gains are shown below.

## Unstable ( pitch gain too low )
<img src="imgs/SimImpulseResponse(200-500N)_Kp0.60_UnstableAt400N.png" alt="Sim Kp0.6"/>

## Better response but still not desirable ( pitch gain still too low )
<img src="imgs/SimImpulseResponse(200-500N)_Kp0.75.png" alt="Sim Kp0.75"/>

## Stable ( pitch gain tuned )
<img src="imgs/SimImpulseResponse(200-500N)_Kp1.0.png" alt="Sim Kp1.0"/>


## Automated Simulation Runs
The above data and plots are generated with a set of automated Python based unit-test scripts. These scripts repeatedly initialize and reset the simulated Bobble-Bot dynamic state, apply an impulse force, allow for and record the controlled response for ten seconds, and then repeat the process with a new impulse force. Once all the data is captured, open-source Python analysis tools [pandas](https://pandas.pydata.org/) and [matplotlib](https://matplotlib.org/) are used to generate the curves shown in the previous section. This is a useful approach that enables using the simulation to perform any number of automated, closed-loop, tests of the balance control logic. The video below shows an example of this in action. It is important to note that the simulator can also be run in a headless state (no 3D graphics). This cuts down on the length of time needed to perform each automated test run.


In [ ]:
from IPython.display import HTML

# Youtube
HTML('<center><iframe width="800" height="500" src="https://www.youtube.com/embed/KSGxodHt-Qk" frameborder="0" allow="autoplay; encrypted-media" allowfullscreen></iframe>')


# Conclusion
The preceding sections of this report cover the use of simulation for the development and test of control algorithms for a two wheeled self-balancing robot. We started with an overview of the real system hardware and its performance. Motivated by a desire to improve controller performance, minimize risk, and reduce cost to ongoing controller development, we decided to create a high fidelity simulator. To start, we derived a mathematical model of our system and arrived at a state space formulation. From there, we surveyed a variety of the commonly selected simulation tools and assesed their strengths and weaknesses. After the pros and cons were considered, the Gazebo simulation environment was selected. Using Gazebo, a simulation of the robot was developed and then used to tune our balance control algorithm. A sampling of results were then recorded and presented. Follow on work will include testing the tuned balance control algorithm with the Bobble-Bot hardware and assessing the performance improvements. Data recorded from these hardware based tests will then feed back into the simulation development and tuning. A few testing iterations will be done until we are confident that the simulated robot and real robot have similar response characteristics. These testing iterations will help to ensure that a robust set of control logic is arrived at. The hope is that Bobble-Bot will then be able to execute user supplied 2D position and velocity profiles. Given our time and budget constraints, it is hard to imagine how this end result would be possible without these simulators.

# References

[1] Lundberg, K. and Barton, T. (2010). History of Inverted-Pendulum Systems. IFAC Proceedings Volumes, 42(24), pp.131-135. 

[2] Learn.adafruit.com. (2018). Overview | Adafruit BNO055 Absolute Orientation Sensor | Adafruit Learning System. [online] Available at: https://learn.adafruit.com/adafruit-bno055-absolute-orientation-sensor/overview [Accessed 18 June 2018].

[3] AG, a. (2018). AS5047D Sensor Product Home - ams AG. [online] Ams.com. Available at: http://ams.com/eng/Products/Magnetic-Position-Sensors/Angle-Position-On-Axis/AS5047D [Accessed 8 July 2018].

[4] Fricke, T. (2012). State Space Intro Talk. Available at: https://github.com/tobin/statespace-intro-talk [Accessed 18 July 2018].

[5] ROS Control Tutorials http://gazebosim.org/tutorials/?tut=ros_control [Accessed 1 July 2018].


## Additional Resources
1. https://matplotlib.org/examples/animation/double_pendulum_animated.html
2. http://www.pages.drexel.edu/~dml46/Tutorials/BalancingBot/files/NXTway-GS%20Model-Based_Design.pdf
3. http://lpsa.swarthmore.edu/Representations/SysRepSS.html
4. https://pdfs.semanticscholar.org/44ca/eb172d89099c184031da6c163c4edd7c8671.pdf
5. https://docs.unrealengine.com/en-us/Engine/Physics
6. https://www.microsoft.com/en-us/research/wp-content/uploads/2017/07/1705.05065.pdf
7. https://www.microsoft.com/en-us/research/project/aerial-informatics-robotics-platform/
8. http://gazebosim.org/tutorials/?tut=ros_control
